<a href="https://colab.research.google.com/github/jvpalhares/IA353-vonzuben-pc2-2023.1/blob/main/PC2_Ativ9_PLN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Notebook #9**
## Encoder-Decoder LSTM for Natural Language Processing.
**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** João Victor Palhares Barbosa


In [1]:
from random import seed
from random import randint
from numpy import array
from numpy import argmax

In [2]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X,y = list(), list()
    for i in range(n_examples):
        in_pattern=[randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X,y

In [3]:
seed(1)
n_samples =1
n_numbers = 2
largest = 10
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

[[3, 10]] [13]


In [4]:
from math import ceil
from math import log10

In [5]:
def to_string(X,y,n_numbers,largest):
    max_length = n_numbers*ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    maxlength = ceil(log10(n_numbers*(largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' 'for _ in range(maxlength-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [6]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

In [7]:
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

X,y = to_string(X,y,n_numbers,largest)
print(X,y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [8]:
alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

In [9]:
def integer_encode(X,y,alphabet):
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [10]:
X,y = integer_encode(X,y,alphabet)

In [11]:
print(X,y)

[[11, 3, 10, 1, 0]] [[1, 3]]


In [12]:
def one_hot_encode(X,y,max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
        
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [13]:
X,y = one_hot_encode(X,y,len(alphabet))

In [14]:
print(X,y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [15]:
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y

In [16]:
def invert(seq,alphabet):
    int_to_char = dict((i,c) for i,c in enumerate(alphabet))
    strings  = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [17]:
n_terms = 3
largest = 10
alphabet = [str(x) for x in range(10)] + ['+', ' ']

In [18]:
n_chars = len(alphabet)
n_in_seq_length = n_terms*ceil(log10(largest+1)) +n_terms-1
n_out_seq_length = ceil(log10(n_terms*(largest+1)))

In [19]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dense

In [20]:
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(n_chars,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                26400     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 75)            0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 50)             25200     
                                                                 
 time_distributed (TimeDistr  (None, 2, 12)            612       
 ibuted)                                                         
                                                                 
Total params: 52,212
Trainable params: 52,212
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
X,y = generate_data(75000,n_terms,largest,alphabet)
model.fit(X,y,epochs=1,batch_size=10)

7500/7500 [==============================] - 60s 7ms/step - loss: 0.4314 - accuracy: 0.8725


In [22]:
X,y = generate_data(100,n_terms,largest,alphabet)
loss,acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.040067, Accuracy: 99.500000


In [23]:
for _ in range(10):
    X,y = generate_data(1,n_terms,largest,alphabet)
    yhat = model.predict(X,verbose=0)
    in_seq = invert(X[0],alphabet)
    out_seq = invert(y[0],alphabet)
    predicted = invert(yhat[0],alphabet)
    print('%s = %s (expect %s)' %(in_seq,predicted,out_seq))

   3+7+5 = 15 (expect 15)
  2+10+6 = 18 (expect 18)
   4+5+9 = 18 (expect 18)
   2+2+4 =  8 (expect  8)
 10+7+10 = 27 (expect 27)
  10+2+3 = 15 (expect 15)
   8+4+1 = 13 (expect 13)
   8+1+3 = 12 (expect 12)
   5+9+6 = 20 (expect 20)
   1+5+8 = 14 (expect 14)
